In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque

# Load trained model (change filename if needed)
net = tf.keras.models.load_model("mnist_cnn_v2.h5")  # replace if needed

# Preprocess ROI to standard MNIST format
def prepare(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # Adaptive threshold: white digit (paper black pen)
    _, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    th = cv2.bitwise_not(th)  # invert: white digit on black

    # Morphology to thicken strokes
    kernel = np.ones((2,2), np.uint8)
    th = cv2.dilate(th, kernel, iterations=2)
    th = cv2.erode(th, kernel, iterations=1)

    # Find largest contour (the digit)
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    digit = th[y:y+h, x:x+w]

    # Resize while maintaining aspect ratio
    if h > w:
        new_h = 20
        new_w = max(int(w * 20 / h), 1)
    else:
        new_w = 20
        new_h = max(int(h * 20 / w), 1)

    resized = cv2.resize(digit, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Place in 28x28 canvas
    canvas = np.zeros((28,28), dtype=np.uint8)
    xoff = (28 - new_w)//2
    yoff = (28 - new_h)//2
    canvas[yoff:yoff+new_h, xoff:xoff+new_w] = resized

    # Center using moments
    M = cv2.moments(canvas)
    if M["m00"] != 0:
        cx, cy = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
        dx, dy = 14 - cx, 14 - cy
        canvas = cv2.warpAffine(canvas, np.float32([[1,0,dx],[0,1,dy]]), (28,28))

    # Normalize 0-1
    norm = canvas.astype("float32") / 255.0
    norm = norm.reshape(1,28,28,1)

    return norm, canvas

# Predict digit from ROI
def predict_digit(img):
    processed = prepare(img)
    if processed is None:
        return -1, 0.0, None
    norm, canvas = processed
    prob = net.predict(norm, verbose=0)[0]
    return np.argmax(prob), np.max(prob), canvas

# Webcam setup
cap = cv2.VideoCapture(0)
memory = deque(maxlen=7)  # smooth predictions

while True:
    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    box = 280
    x1, y1 = W//2 - box//2, H//2 - box//2
    x2, y2 = x1 + box, y1 + box

    roi = frame[y1:y2, x1:x2]
    d, conf, canvas = predict_digit(roi)
    memory.append((d, conf))

    # Majority voting
    votes = np.zeros(10)
    for digit, score in memory:
        if digit != -1:
            votes[digit] += score
    if votes.sum() > 0:
        d = votes.argmax()
        conf = votes[d] / votes.sum()
    else:
        d = -1
        conf = 0

    # Display results
    cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), 2)
    msg = f"Digit: {d} ({conf*100:.1f}%)" if d != -1 else "No digit"
    cv2.putText(frame, msg, (20,50), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0,0,255),2)

    cv2.imshow("Digit Detector", frame)

    if canvas is not None:
        # Scale 28x28 canvas to 280x280 for better visibility
        canvas_large = cv2.resize(canvas, (280,280), interpolation=cv2.INTER_NEAREST)
        cv2.imshow("28x28 Input to Model", canvas_large)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
